# Solving elasticity problems using JuliaFEM

Author(s): Jukka Aho

**Abstract**: A workflow to solve typical elasticity problem. This document also tries to give some quidelines how to develop JuliaFEM.

## Bottom-up design

We go piece by piece starting from something simple and going up to more complicated programming model.

*Design principle 1*: we introduce new ideas using Notebooks.

*Design principle 2*: we use ``Logging``. Forget ``println``.

In [1]:
using JuliaFEM
using Logging
Logging.configure(level=DEBUG)

Logger(root,DEBUG,Pipe(open, 0 bytes waiting),root)

 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in int32 at deprecated.jl:49
 in recv at /Users/jukka/.julia/v0.4/ZMQ/src/ZMQ.jl:617
 in recv_ipython at /Users/jukka/.julia/v0.4/IJulia/src/msg.jl:63
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:120
 in anonymous at task.jl:365
while loading /Users/jukka/.julia/v0.4/IJulia/src/kernel.jl, in expression starting on line 35
 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in int32 at deprecated.jl:49
 in recv at /Users/jukka/.julia/v0.4/ZMQ/src/ZMQ.jl:617
 in recv_ipython at /Users/jukka/.julia/v0.4/IJulia/src/msg.jl:63
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:120
 in anonymous at task.jl:365
while loading /Users/jukka/.julia/v0.4/IJulia/src/kernel.jl, in expression starting on line 35


*Design principle 3*: we write docstrings using [numpy style](https://github.com/numpy/numpy/blob/master/doc/HOWTO_DOCUMENT.rst.txt).

*Design principle 4*: we don't use greek characters in code which is implemented to JuliaFEM. In notebooks they are ok.

*Design principle 5*: we use 4 space indentation like in Python.

Our task is: for given $\mathbf{u}$ calculate $\mathbf{R}(\mathbf{u}) = \mathbf{T}(\mathbf{u}) - \mathbf{F}(\mathbf{u})$ and it's partial derivative with respect to $\mathbf{u}$, i.e. $\partial \mathbf{R}(\mathbf{u}) / \partial \mathbf{u}$.

In [2]:
function Wint_integrand(el, ip)
    xi = ip.xi
    dbasisdX = JuliaFEM.get_dbasisdX(el, ip)
    u = el.attributes["displacement"]

    # kinematics
    gradu = u*dbasisdX
    F = I + gradu
    E = 1/2*(gradu' + gradu + gradu'*gradu)

    # constitutive equation
    lambda = JuliaFEM.interpolate(el, "lambda", xi)
    mu = JuliaFEM.interpolate(el, "mu", xi)
    S = lambda*trace(E)*I + 2*mu*E
    P = F*S
    T = P*dbasisdX'
    return T
end

Wint = JuliaFEM.integrate(Wint_integrand)

# calculate partial derivatives of R with respect to field "displacement"
jacobian = JuliaFEM.linearize(Wint, "displacement")

jacobian (generic function with 1 method)

That was our geometrically nonlinear elasticity solver. Note how we used automatic differentiation to linearize residual vector.

*Design principle 6*: we test our code. We use FactCheck for testing.

In [3]:
using FactCheck

In [4]:
function get_test_element()
    # set up one linear quadrangle element
    basis(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]
    dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
                   (1-xi[2])/4.0    -(1+xi[1])/4.0
                   (1+xi[2])/4.0     (1+xi[1])/4.0
                  -(1+xi[2])/4.0     (1-xi[1])/4.0]
    integration_points = [
        JuliaFEM.IntegrationPoint(1.0/sqrt(3.0)*[-1, -1], 1.0, Dict()),
        JuliaFEM.IntegrationPoint(1.0/sqrt(3.0)*[ 1, -1], 1.0, Dict()),
        JuliaFEM.IntegrationPoint(1.0/sqrt(3.0)*[ 1,  1], 1.0, Dict()),
        JuliaFEM.IntegrationPoint(1.0/sqrt(3.0)*[-1,  1], 1.0, Dict())]
    attributes = Dict()
    element_id = 1
    node_ids = [1, 2, 3, 4]
    e = JuliaFEM.Element(element_id, node_ids, basis, dbasis, integration_points, attributes)
    E = 90.0
    nu = 0.25
    mu = E/(2*(1+nu))
    la = E*nu/((1+nu)*(1-2*nu))
    la = 2*la*mu/(la + 2*mu)
    e.attributes["coordinates"] = [0.0 0.0; 10.0 0.0; 10.0 1.0; 0.0 1.0]'
    e.attributes["lambda"] = la
    e.attributes["mu"] = mu
    e.attributes["displacement"] = [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]'
    return e
end

facts("test solve one element model") do

    e = get_test_element()
    F = [0.0 0.0; 0.0 0.0; 0.0 -2.0; 0.0 0.0]'

    du = zeros(2, 4)

    free_dofs = [3, 4, 5, 6]
    for i=1:10
        R = Wint(e)
        K = jacobian(e)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        e.attributes["displacement"] += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end

    # Tested against Elmer solution
    u = e.attributes["displacement"]
    Logging.debug("solution vector: \n $u")
    @fact u[2, 3] --> roughly(-2.222244754401764)
    norm1 = norm(u)
    Logging.debug("norm of u: $(norm(u))")

    # We rotate model a bit and make sure that norm remains same
    phi = 30/180*pi
    rmat = [
        cos(phi) -sin(phi)
        sin(phi)  cos(phi)]
    e.attributes["coordinates"] = rmat*e.attributes["coordinates"]
    F = rmat*F

    e.attributes["displacement"] = [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]'
    du = zeros(2, 4)
    for i=1:10
        R = Wint(e)
        K = jacobian(e)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        e.attributes["displacement"] += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    u = e.attributes["displacement"]
    Logging.debug("solution vector: \n $u")
    Logging.debug("norm of u: $(norm(u))")
    @fact norm(u) --> roughly(norm1) 
end

test solve one element model


19-Aug 18:39:44:DEBUG:root:Converged in 6 iterations.
19-Aug 18:39:44:DEBUG:root:solution vector: 
 [0.0 -0.3991450609547433 -0.07228582695592461 0.0
 0.0 -2.1779892317073504 -2.222244754401764 0.0]
19-Aug 18:39:44:DEBUG:root:norm of u: 3.1292483947150047
19-Aug 18:39:45:DEBUG:root:Converged in 6 iterations.
19-Aug 18:39:45:DEBUG:root:solution vector: 
 [0.0 0.7433248532717796 1.048521014723486 0.0
 0.0 -2.085766534304891 -1.9606633242166027 0.0]
19-Aug 18:39:45:DEBUG:root:norm of u: 3.1292483947150056


2 facts verified.


delayed_handler (generic function with 4 methods)

In [5]:
function assemble_element!(ass::JuliaFEM.Assembly, el::JuliaFEM.Element)

    gdofs = ass.gdofs[el.id]
    R = Wint(el)
    K = jacobian(el)
    dofs = length(R)

    for i=1:dofs
        for j=1:dofs
            push!(ass.I, gdofs[i])
            push!(ass.J, gdofs[j])
            push!(ass.A, K[i,j])
        end
        push!(ass.i, gdofs[i])
        push!(ass.b, R[i])
    end
end

assemble_element! (generic function with 1 method)

Time to test again. From last test we know that correct solution is

    [0.0 -0.39914506095474317 -0.07228582695592449 0.0
     0.0 -2.1779892317073504  -2.222244754401764   0.0]

This time we assemble global stiffness matrix in different order, 2 3 4 1

In [6]:
facts("one element assembly") do
    # Create model
    Logging.debug("Adding nodes to array")
    el = get_test_element()
    
    for i=1:10
        Logging.debug("Starting iteration $i")
        ass = JuliaFEM.Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], Dict{Int64,Array{Int64,1}}())
        ass.gdofs[el.id] = [1, 2, 3, 4, 5, 6, 7, 8]
        Logging.debug("Assembling")
        assemble_element!(ass, el)

        # Boundary conditions
        F = [0.0 0.0; 0.0 0.0; 0.0 -2.0; 0.0 0.0]'
        F = F[:]
        free_dofs = [3, 4, 5, 6]

        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F
        du = zeros(8)
        du[free_dofs] = K[free_dofs, free_dofs] \ -R[free_dofs]

        Logging.debug("Solution norm = $(norm(du))")

        # update solution back to elements
        eldu = du[ass.gdofs[el.id]]
        eldu = reshape(eldu, (2, round(Int, length(eldu)/2)))
        el.attributes["displacement"] += eldu
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    disp = el.attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    @fact norm(disp) --> roughly(3.1292483947150043)
end

one element assembly


19-Aug 18:39:46:DEBUG:root:Adding nodes to array
19-Aug 18:39:46:DEBUG:root:Starting iteration 1
19-Aug 18:39:46:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 3.0900221367289986
19-Aug 18:39:47:DEBUG:root:Starting iteration 2
19-Aug 18:39:47:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 0.3212131602153472
19-Aug 18:39:47:DEBUG:root:Starting iteration 3
19-Aug 18:39:47:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 0.04043178193999703
19-Aug 18:39:47:DEBUG:root:Starting iteration 4
19-Aug 18:39:47:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 0.0009291101052105739
19-Aug 18:39:47:DEBUG:root:Starting iteration 5
19-Aug 18:39:47:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 1.5638899027804743e-7
19-Aug 18:39:47:DEBUG:root:Starting iteration 6
19-Aug 18:39:47:DEBUG:root:Assembling
19-Aug 18:39:47:DEBUG:root:Solution norm = 1.0464940956129567e-14
19-Aug 18:39:47:DEBUG:root:Converged in 6 iterations

1 fact verified.


delayed_handler (generic function with 4 methods)

Seems to be working. But we still need to handle boundary conditions more "cleverly" and generalize assembly to several elements (which is not problem).

First of all, essential boundary conditions are nothing more than equality constraints saying that value for some degree of freedom is fixed. Elimination is just a special case when this value equals to zero. There is couple of different strategies to handle essential boundary conditions. One option is to force them using Lagrange multipliers which can also be used to create all kind of kinematic constraints also. (For example, contact can be considered as a kinematic constraint.) Another option is to manipulate matrix such a way that constraint is satisfied.

Because we are now going "bottom-up", we develop something extremely simple that however deals with the problem:

In [7]:
type BC
    dofs :: Array{Int64, 1}
    values :: Array{Float64, 1}
end

In [8]:
"""
Create local dof to global dof mapping for given elements
"""
function create_ldof2gdofmap(elements, field)

    ndofs = size(elements[1].attributes[field], 1)
    
    all_node_ids = Int64[]
    for el in elements
        for nid in el.node_ids
            push!(all_node_ids, nid)
        end
    end
    all_node_ids = unique(all_node_ids)

    # Assign global dof for each node
    pdim = 1
    ngdofs = Dict{Int64, Array{Int64,1}}()
    for nid in all_node_ids
        ngdofs[nid] = collect(pdim:pdim+ndofs-1)
        pdim += ndofs
    end

    return ngdofs
end

create_ldof2gdofmap (generic function with 1 method)

In [16]:
function solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=2, max_iterations=10)

    Logging.info("solve!: dofs per node: $ndofs")
    pdim = length(dofmap)*ndofs
    Logging.debug("Problem size = $pdim")

    # Assign global dofs for elements
    gdofs = Dict{Int64, Array{Int64,1}}()
    for el in elements
        gdofs[el.id] = Int64[]
        for nid in el.node_ids
            for ndof in dofmap[nid]
                push!(gdofs[el.id], ndof)
            end
        end
    end

    ass = JuliaFEM.Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], gdofs)

    for iter=1:max_iterations
        Logging.debug("Starting iteration $iter")
        ass.I = []
        ass.J = []
        ass.A = []
        ass.i = []
        ass.b = []
        
        Logging.debug("Assembling")
        for el in elements
            assemble_element!(ass, el)
        end

        i = 1
        Logging.debug("Adding Dirichlet boundary conditions using Lagrange multipliers")
        # Dirichlet boundary conditions
        for bc in dirichlet_bcs
            for (dof, val) in zip(bc.dofs, bc.values)
                #Logging.debug("dof $dof => $val")
                push!(ass.I, dof)
                push!(ass.J, pdim+i)
                push!(ass.A, 1)
                push!(ass.I, pdim+i)
                push!(ass.J, dof)
                push!(ass.A, 1)
                push!(ass.i, pdim+i)
                push!(ass.b, 0)
                i += 1
            end
        end
        Logging.debug("Added $i Lagrange multipliers to model")
        i -= 1

        Logging.debug("Adding Neumann boundary conditions")
        F = zeros(pdim+i)
        # Neumann boundary conditions
        for bc in neumann_bcs
            for (dof, val) in zip(bc.dofs, bc.values)
                F[dof] += val
            end
        end

        Logging.debug("Solving system of equations. Total size = $(pdim+i)")
        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F

        du = K \ -R

        #du = reshape(du, 2, 6)
        solnorm = norm(du[1:pdim])
        #Logging.debug("du = $du")
        Logging.debug("Solution norm du = $solnorm")

        # update solution back to elements
        for el in elements
            #Logging.debug("update element $(el.id)")
            eldisp = el.attributes["displacement"]
            #Logging.debug("displacement before update $(el.id) : \n$eldisp")
            eldu = du[ass.gdofs[el.id]]
            eldu = reshape(eldu, (ndofs, round(Int, length(eldu)/ndofs)))
            #Logging.debug("eldu for element $(el.id) \n$eldu")
            el.attributes["displacement"] += eldu
            eldisp = el.attributes["displacement"]
            #Logging.debug("displacement after update $(el.id) : \n$eldisp")

        end
        if solnorm < 1.0e-9
            Logging.debug("Converged in $iter iterations.")
            break
        end
    end

end

ENV["COLUMNS"] = 160

facts("solve one element problem") do
    # Create model
    el = get_test_element()
    elements = [el]
    # Initialize elements ready for solution
    for el in elements
        eldim, elnodes = size(el.attributes["coordinates"])
        el.attributes["displacement"] = zeros(2, elnodes)
    end
    dofmap = create_ldof2gdofmap(elements, "displacement")
    Logging.debug(dofmap)
    # Boundary conditions
    # here we want to create nodal force for third dof, that is, node id 2, second dof
    bc1 = BC([dofmap[3][2]], [-2.0])
    # dirichlet bc, set dx=dy=0 on support
    bc2 = BC([dofmap[1][1], dofmap[1][2], dofmap[4][1], dofmap[4][2]], [0.0, 0.0, 0.0, 0.0])
    solve!(elements, dofmap, [bc1], [bc2]; max_iterations=7)
    disp = elements[1].attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    @fact norm(disp) --> roughly(3.1292483947150043)
end


solve one element problem


19-Aug 18:49:12:DEBUG:root:Dict(4=>[7,8],2=>[3,4],3=>[5,6],1=>[1,2])
19-Aug 18:49:13:INFO:root:solve!: dofs per node: 2
19-Aug 18:49:13:DEBUG:root:Problem size = 8
19-Aug 18:49:13:DEBUG:root:Starting iteration 1
19-Aug 18:49:13:DEBUG:root:Assembling
19-Aug 18:49:13:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
19-Aug 18:49:13:DEBUG:root:Added 5 Lagrange multipliers to model
19-Aug 18:49:13:DEBUG:root:Adding Neumann boundary conditions
19-Aug 18:49:13:DEBUG:root:Solving system of equations. Total size = 12
19-Aug 18:49:13:DEBUG:root:Solution norm du = 3.0900221367289444
19-Aug 18:49:13:DEBUG:root:Starting iteration 2
19-Aug 18:49:13:DEBUG:root:Assembling
19-Aug 18:49:13:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
19-Aug 18:49:13:DEBUG:root:Added 5 Lagrange multipliers to model
19-Aug 18:49:13:DEBUG:root:Adding Neumann boundary conditions
19-Aug 18:49:13:DEBUG:root:Solving system of equations. Total size = 12
19-Aug 18:49:13:DEB

1 fact verified.


delayed_handler (generic function with 4 methods)

## Top-down design

Next we see this problem from "other direction", by parsing ABAQUS .inp file and making 3d simulation.

In [17]:
using JuliaFEM.abaqus_reader
fid = open("../geometry/3d_beam/palkki.inp")
model = JuliaFEM.abaqus_reader.parse_abaqus(fid)
close(fid)
model

19-Aug 18:49:21:INFO:root:Registered handlers: Any["ELEMENT","NODE","NSET"]


LoadError: LoadError: MethodError: `convert` has no method matching convert(::Type{SubString{ASCIIString}}, ::Dict{Any,Any})
This may have arisen from a call to the constructor SubString{ASCIIString}(...),
since type constructors fall back to convert methods.
Closest candidates are:
  call{T}(::Type{T}, ::Any)
  convert{T<:AbstractString}(::Type{T<:AbstractString}, !Matched::AbstractArray{Char,1})
  convert{T<:AbstractString}(::Type{SubString{T<:AbstractString}}, !Matched::T<:AbstractString)
  ...
while loading In[17], in expression starting on line 3

 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in beginswith at deprecated.jl:30
 in parse_abaqus at /Users/jukka/.julia/v0.4/JuliaFEM/src/abaqus_reader.jl:115
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[17], in expression starting on line 3
 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in beginswith at deprecated.jl:30
 in parse_abaqus at /Users/jukka/.julia/v0.4/JuliaFEM/src/abaqus_reader.jl:115
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[17], in expression starting on line 3


In [14]:
function solve_3d_model()
    Logging.debug("Creating elements")
    elements = Element[]
    coordinates = zeros(3, 10)
    for (elid, node_ids) in model["elements"]
        for (i, nid) in enumerate(node_ids)
            coordinates[:,i] = model["nodes"][nid]
        end
        attributes = Dict("Young" => 90, "Poisson" => 0.25)
        el = Element(elid, node_ids, coordinates, attributes)
        push!(elements, el)
    end

    # create "dofmap" so that we know how to assemble global stiffness matrix
    dofmap = create_ldof2gdofmap(elements; ndofs=3)

    # Boundary conditions

    # dirichlet bc, set dx=dy=dz for all nodes in set SUPPORT
    bc_support = BC(Int64[], Float64[])
    for nid in model["nsets"]["SUPPORT"]
        for i=1:3
            push!(bc_support.dofs, dofmap[nid][i])
            push!(bc_support.values, 0.0)
        end
    end

    # force boundary condition, put -1 to 2nd dof for each node in set LOAD
    bc_load = BC(Int64[], Float64[])
    for nid in model["nsets"]["LOAD"]
        push!(bc_load.dofs, dofmap[nid][2])
        push!(bc_load.values, -30.0)
    end

    #solve!(elements, [bc1], [bc2]; max_iterations=7)
    neumann_bcs = [bc_load]
    dirichlet_bcs = [bc_support]
    # ndofs = dimension of unknown field in nodes
    solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=3, max_iterations=10)

    # Let's pick maximum absolute displacement in y direction
    maxdisp = 0.0
    for el in elements
        eldisp = el.attributes["displacement"]
        eldispy = eldisp[2,:]
        maxeldisp = maximum(abs(eldispy))
        if maxeldisp > maxdisp
            maxdisp = maxeldisp
        end
    end
    Logging.info("Maximum absolute displacement in y direction: $maxdisp")
    return model, elements, dofmap
end

model, elements, dofmap = solve_3d_model();

15-Aug 17:04:30:DEBUG:root:Creating elements
15-Aug 17:04:31:INFO:root:create_ldof2gdofmap: dofs per node: 3
15-Aug 17:04:31:INFO:root:solve!: dofs per node: 3
15-Aug 17:04:31:DEBUG:root:Problem size = 894
15-Aug 17:04:31:DEBUG:root:Starting iteration 1
15-Aug 17:04:31:DEBUG:root:Assembling
15-Aug 17:04:36:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
15-Aug 17:04:36:DEBUG:root:Added 28 Lagrange multipliers to model
15-Aug 17:04:36:DEBUG:root:Adding Neumann boundary conditions
15-Aug 17:04:36:DEBUG:root:Solving system of equations. Total size = 921
15-Aug 17:04:36:DEBUG:root:Solution norm du = 72.87727091053921
15-Aug 17:04:36:DEBUG:root:Starting iteration 2
15-Aug 17:04:36:DEBUG:root:Assembling
15-Aug 17:04:40:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
15-Aug 17:04:40:DEBUG:root:Added 28 Lagrange multipliers to model
15-Aug 17:04:40:DEBUG:root:Adding Neumann boundary conditions
15-Aug 17:04:40:DEBUG:root:Solving system of eq

## Saving results to file

In [15]:
xdoc, xmodel = JuliaFEM.xdmf.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.xdmf.xdmf_new_grid(temporal_collection; time=0)

<Grid Name="Grid">
  <Time Value="0"/>
</Grid>


Save geometry to xdmf file

In [16]:
nnodes = length(model["nodes"])
Logging.info("Number of nodes in model: $nnodes")
node_ids = Int64[]
for nid in keys(model["nodes"])
    push!(node_ids, nid)
end
sort!(node_ids)
X = zeros(3, nnodes)
for (i, nid) in enumerate(node_ids)
    X[:,i] = model["nodes"][nid]
end

nelements = length(model["elements"])
Logging.info("Number of elements in model: $nelements")
elmap = zeros(Int64, 11, nelements)
elmap[1,:] = 0x0026
for elid in 1:nelements
    elmap[2:end,elid] = model["elements"][elid]
end

15-Aug 17:04:54:INFO:root:Number of nodes in model: 298
15-Aug 17:04:54:INFO:root:Number of elements in model: 120


In [17]:
using LightXML

function xdmf_new_mesh(grid, X, elmap)
    dim, nnodes = size(X)
    geometry = new_child(grid, "Geometry")
    set_attribute(geometry, "Type", "XYZ")
    dataitem = new_child(geometry, "DataItem")
    set_attribute(dataitem, "DataType", "Float")
    set_attribute(dataitem, "Dimensions", "$nnodes $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    #add_text(dataitem, join(X, " "))
    s = "\n"
    
    for i=1:nnodes
        s *= "\t\t" * join(X[:,i], " ") * "\n"
    end
    s *= "       "
    add_text(dataitem, s)

    elmap2 = copy(elmap)
    elmap2[2:end,:] -= 1
    dim, nelements = size(elmap2)

    topology = new_child(grid, "Topology")
    #set_attribute(topology, "Dimensions", "1")
    set_attribute(topology, "TopologyType", "Mixed")
    set_attribute(topology, "NumberOfElements", nelements)
    dataitem = new_child(topology, "DataItem")
    set_attribute(dataitem, "DataType", "Int")
    set_attribute(dataitem, "Dimensions", "$nelements $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    s = "\n"
    for i=1:nelements
        s *= "\t\t" * join(elmap2[:,i], " ") * "\n"
    end
    add_text(dataitem, s)
    #add_text(dataitem, join(elmap2, " "))
    
end

xdmf_new_mesh(grid, X, elmap)

true

In [18]:
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/3d_solid_model.xmf")

10435

Save nodal data to model

In [19]:
nodaldisp = Dict()
for el in elements
    for (i, nid) in enumerate(el.node_ids)
        nodaldisp[nid] = el.attributes["displacement"][:, i]
    end
end
nodaldisp

Dict{Any,Any} with 298 entries:
  288 => [-0.21784897897079505,-6.64256844921929,0.18339325827190953]
  11  => [-0.17342410820059667,-6.343125806991628,0.17544367431766225]
  158 => [-0.0007723334905839533,-0.2321904799732306,-0.03883691791045858]
  215 => [-0.03299227384130259,-4.2236276356937505,-0.0017757488281830807]
  134 => [-0.06336705102703495,-3.3332586850719337,0.06555932581531249]
  160 => [0.03205303832663507,-0.536361456950779,-0.07291283371223893]
  29  => [-0.01620524134284992,-0.19455583841608232,-0.03838421619811468]
  131 => [0.04015396763870081,-3.5101776818531523,0.10045744211179368]
  249 => [-0.2491004521409732,-6.570813533283954,0.06837890174506163]
  207 => [0.03974407267311282,-3.48588780058722,0.007971846403163977]
  173 => [-0.04472757590353555,-4.582185697159958,-0.04434348606400347]
  289 => [-0.16494613158592017,-6.360857515886982,0.33435404817411213]
  74  => [-0.03449870789750817,-3.947284664599636,0.0692691027146262]
  201 => [-0.00128347996939122,-3.85

In [20]:
u = zeros(3, nnodes)
for (i, nid) in enumerate(node_ids)
    u[:,i] = nodaldisp[nid]
end
u

3x298 Array{Float64,2}:
 -0.202475   -0.17437     0.0131267   -0.0340868  -0.0252162   0.0521758  …  -0.054781    -0.169903   0.186936   0.102666  -0.0970747  -0.164397  0.0
 -1.34204    -1.66404    -3.20587     -3.27673    -3.68732    -3.36789       -4.33822     -1.03943   -2.27466   -1.67467   -2.91213    -2.52353   0.0
  0.0211856   0.0324162   0.00213878   0.0377156   0.0683502   0.030081      -0.00684333  -0.035509   0.304867   0.26033    0.113802    0.152433  0.0

In [21]:
JuliaFEM.xdmf.xdmf_new_field(grid, "Displacement", "nodes", u)

true

In [22]:
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/3d_solid_model.xmf")

28330

In [17]:
u = zeros(10)
u1 = u[1:5]
u, u1

([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[0.0,0.0,0.0,0.0,0.0])

In [18]:
u1[:] = 1
u, u1

([0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0],[1.0,1.0,1.0,1.0,1.0])